# 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기

In [ ]:
df_standard_train = pd.read_csv('/content/drive/MyDrive/hana1/Team/final/drop_standard_test.csv', index_col = 0)
df_standard_train.shape

(14007, 23)

# 데이터 분할

In [ ]:
# X, y 분할
X_train = df_standard_train.drop('Credit_Score', axis = 1)

print("X_train 데이터: ", X_train.shape)

X_train 데이터:  (14007, 22)


# Model 2 생성

## K-means

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=2, random_state=42)
km.fit(X_train)
print(km.labels_)
print(np.unique(km.labels_, return_counts=True))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[0 0 0 ... 0 0 0]
(array([0, 1], dtype=int32), array([9412, 4595]))


In [ ]:
df_X_train = X_train.copy()
df_X_train['Cluster'] = km.labels_
df_X_train

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Cluster
1,10,24.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,...,1,809.98,33.053114,274.0,0,49.574949,21.465380,1,361.444004,0
2,11,24.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,-1.0,...,1,809.98,33.811894,274.0,0,49.574949,148.233938,4,264.675446,0
3,12,24.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,4.0,...,1,809.98,32.430559,276.0,0,49.574949,39.082511,1,343.826873,0
8,9,35.0,4,143162.64,12187.220000,1.0,5.0,8.0,3.0,8.0,...,1,1303.01,35.229707,221.0,0,246.992319,397.503654,4,854.226027,1
9,10,35.0,4,143162.64,12187.220000,1.0,5.0,8.0,3.0,6.0,...,1,1303.01,35.685836,222.0,0,246.992319,453.615131,3,788.114550,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49975,12,33.0,10,59146.36,4908.863333,3.0,6.0,6.0,1.0,8.0,...,1,418.03,26.830861,252.0,0,26.778419,210.331375,5,543.776539,0
49992,9,29.0,1,20002.88,1929.906667,10.0,8.0,29.0,5.0,33.0,...,0,3571.70,32.391288,76.0,1,60.964772,107.210742,5,314.815153,0
49993,10,29.0,1,20002.88,1929.906667,10.0,8.0,29.0,5.0,33.0,...,0,3571.70,37.528511,77.0,1,60.964772,71.794421,5,350.231473,0
49994,11,29.0,1,20002.88,1929.906667,10.0,8.0,29.0,5.0,33.0,...,0,3571.70,27.027812,78.0,1,60.964772,50.846847,2,341.179047,0


In [ ]:
clusters_train = df_X_train.groupby('Cluster')

df_clusters_mean_train = pd.DataFrame(clusters_train.mean())
df_clusters_mean_train = df_clusters_mean_train.T
df_clusters_mean_train

Cluster,0,1
Month,10.497556,10.507726
Age,34.119422,35.159521
Occupation,6.893328,7.072688
Annual_Income,28815.115452,94375.092416
Monthly_Inhand_Salary,2392.611345,7859.290216
Num_Bank_Accounts,5.236507,4.729924
Num_Credit_Card,5.258075,4.963221
Interest_Rate,12.890034,11.235909
Num_of_Loan,3.040055,2.408487
Delay_from_due_date,17.970251,15.417410


### 변수 중요도

In [ ]:
# 각 클러스터의 중심(평균) 계산
cluster_centers = km.cluster_centers_

# 변수 중요도 추정
variable_importance = cluster_centers.std(axis=0)

print("변수 중요도:")
print(variable_importance)

변수 중요도:
[5.40712108e-03 5.23022708e-01 9.16183613e-02 3.27763516e+04
 2.73286494e+03 2.54552974e-01 1.47122575e-01 8.30229101e-01
 3.16307070e-01 1.27682671e+00 1.54845016e+00 6.33116147e-01
 1.92332093e+00 1.37777676e-02 1.08998623e+02 7.05878217e-01
 1.39515038e+01 5.90606844e-02 3.55583029e+01 1.01108712e+02
 4.79066235e-01 1.29482869e+02]


In [ ]:
df_clusters_mean_train['variable_importance'] = variable_importance
df_clusters_mean_train

Cluster,0,1,variable_importance
Month,10.497556,10.507726,0.005407
Age,34.119422,35.159521,0.523023
Occupation,6.893328,7.072688,0.091618
Annual_Income,28815.115452,94375.092416,32776.351602
Monthly_Inhand_Salary,2392.611345,7859.290216,2732.864941
Num_Bank_Accounts,5.236507,4.729924,0.254553
Num_Credit_Card,5.258075,4.963221,0.147123
Interest_Rate,12.890034,11.235909,0.830229
Num_of_Loan,3.040055,2.408487,0.316307
Delay_from_due_date,17.970251,15.417410,1.276827


# 데이터 저장

In [ ]:
df_semi_good_train = df_X_train[df_X_train['Cluster']==1]
df_semi_good_train.rename(columns={'Cluster':'Semi_Good'})
df_semi_good_train.shape

(4595, 23)

In [ ]:
df_semi_good_train.to_csv('/content/drive/MyDrive/hana1/Team/final/semi_good_train.csv')

In [ ]:
df_semi_poor_train = df_X_train[df_X_train['Cluster']==0]
df_semi_poor_train.rename(columns={'Cluster':'Semi_Poor'})
df_semi_poor_train.shape

(9412, 23)

In [ ]:
df_semi_poor_train.to_csv('/content/drive/MyDrive/hana1/Team/final/semi_poor_train.csv')